In [1]:
from model import FinancialLSTMModel
import pandas as pd

import os, random, numpy as np, tensorflow as tf

from tensorflow.keras.layers import (
    InputLayer,
    LSTM,
    LayerNormalization,
    Dense,
    BatchNormalization,
    Dropout
)
from tensorflow.keras import optimizers

2025-11-23 21:00:43.124843: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-23 21:00:43.218774: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 21:00:45.080870: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [ ]:
CSV_PATHS = [
    './../data/GOOGL_4h.csv', 
    './../data/AAPL_4h.csv', 
    './../data/^NDX_4h.csv', 
    './../data/^GSPC_4h.csv'
    ]
 
DATE_COL = 'Datetime'
TARGET = 'direction'

SEQ_LENGTH = 60
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 100
TEST_RATIO = 0.1
VAL_SPLIT = 0.1

REPS = 30
OUTPUT_DIR='./../res/'

In [3]:
FEATURES = [
    [("Close", "minmax")],
    
    [("Close", "minmax"),
     ("rsi_14", "minmax"),
     ("rsi_28", "minmax"),
     ("rsi_7", "minmax"),],
    
    [("Close", "minmax"),
     ("macd", "standard"),
     ],
    
    [("Close", "minmax"),
     ("ema_20", "standard"),
     ("ema_50", "standard"),
     ],

    [("Close", "minmax"),
     ("stoch_k", "minmax"),
     ("stoch_d", "minmax"),
     ],

    [("Close", "minmax"),
     ("roc", "standard"),
     ],

    [("Close", "minmax"),
     ("adx", "minmax"),
     ("di_plus", "minmax"),
     ("di_minus", "minmax"),
     ],

    [("Close", "minmax"),
     ("atr", "standard"),
     ],

    [("Close", "minmax"),
     ("close_pos", "none"),
     ],


    [("Close", "minmax"),
     ("body_range_ratio", "none"),
     ],


    [("Close", "minmax"),   
     ("Volume", "minmax"),
     ],
]

In [4]:
HIDDEN_LAYERS = [
        tf.keras.layers.LSTM(96, return_sequences=True, recurrent_dropout=0.1),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.LSTM(96, return_sequences=False, recurrent_dropout=0.1),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
    ]

2025-11-23 21:00:46.094487: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
for file in CSV_PATHS:
    if not os.path.exists(file):
        raise FileNotFoundError(f"File {file} does not exist.")

In [ ]:
res = []
for file in CSV_PATHS:
    for idx, feat in enumerate(FEATURES):
        for r in range(REPS):
            model = FinancialLSTMModel(
                csv_path=file,
                features_scales=feat,
                target_col="direction",
                datetime_col="Datetime",
                
                seq_length=SEQ_LENGTH,
                batch_size=BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                epochs=EPOCHS,
                test_ratio=TEST_RATIO,
                val_split=VAL_SPLIT,
            )

            print(f" ---- Running file {file} with features: {feat}, repetition {r+1}/{REPS} ---- ")

            model.prepare_data()
            model.build_model(HIDDEN_LAYERS)
            model.train()
            ev = model.evaluate()
            print(f" > Evaluation results: {ev}")
            
            res.append({
                "file_path": file,
                "feature_set_index": idx,
                "features": [f[0] for f in feat],
                "repetition": r,
                **ev
            })
            
            df_res = pd.DataFrame(res)
            df_res.to_csv(os.path.join(OUTPUT_DIR, 'model_testing_feature_sets_results.csv'), index=False)
        
        

 ---- Running file ./../data/GOOGL_4h.csv with features: [('Close', 'minmax')], repetition 1/30 ---- 
